In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set working directory
import os
current_dir = os.getcwd()
os.chdir(current_dir.replace('\code', '\data'))

<>:9: SyntaxWarning: invalid escape sequence '\c'
<>:9: SyntaxWarning: invalid escape sequence '\d'
<>:9: SyntaxWarning: invalid escape sequence '\c'
<>:9: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Jack\AppData\Local\Temp\ipykernel_43412\2141329615.py:9: SyntaxWarning: invalid escape sequence '\c'
  os.chdir(current_dir.replace('\code', '\data'))
C:\Users\Jack\AppData\Local\Temp\ipykernel_43412\2141329615.py:9: SyntaxWarning: invalid escape sequence '\d'
  os.chdir(current_dir.replace('\code', '\data'))


In [81]:
# Load data
df = pd.read_csv('2024_wk7_td_odds.csv', delimiter=',')

# Remove rows that contain anything in this string list
string = ['@', 'Any Time', 'Touchdown', 'scorer', 'Sun', ':', 'touchdown', 'Scorer', 'Show', 'Show less', 'ETMore', 'wagers']
df = df[~df['header'].str.contains('|'.join(string))]

# name which is the header column value if the value does not contains a digit
name = df['header'].apply(lambda x: x if not any(i.isdigit() for i in x) else np.nan)
name = name.dropna().reset_index(drop=True)

# odds which is the header column if the header column value contains a digit
value = df['header'].apply(lambda x: x if any(i.isdigit() for i in x) else np.nan)
value = value.astype(float)
value = value.dropna().reset_index(drop=True)

# Concat name and odds columns
odds_df = pd.DataFrame({'name': name, 'odds': value})

def moneyline_to_probability(odds):
    if odds > 0:
        return 100 / (odds + 100)
    else:
        return abs(odds) / (abs(odds) + 100)

# Convert moneyline odds to probability
odds_df['probability'] = odds_df['odds'].apply(moneyline_to_probability)

# Percentile for each probability
odds_df['odds_percentile'] = odds_df['probability'].rank(pct=True)

# Sort by odds
odds_df.sort_values(by='odds', ascending=True).reset_index(drop=True).head()

,name,odds,probability,odds_percentile
0,Kyren Williams,-240.0,0.705882,1.000000
1,Derrick Henry,-185.0,0.649123,0.997347
2,Saquon Barkley,-165.0,0.622642,0.994695
3,Brian Robinson Jr.,-160.0,0.615385,0.992042
4,J.K. Dobbins,-155.0,0.607843,0.989390


In [82]:
# Load projections
projections = pd.read_csv('2024_wk7_pff_proj.csv', delimiter=',')
projections = projections[['playerName', 'rushTd', 'recvTd', 'dstReturnTd']]
projections['totalTd'] = projections['rushTd'] + projections['recvTd'] + projections['dstReturnTd']

# Percentile for totalTD
projections['proj_percentile'] = projections['totalTd'].rank(pct=True)
projections.head()

,playerName,rushTd,recvTd,dstReturnTd,totalTd,proj_percentile
0,Jordan Mason,0.5553,0.1795,0.0,0.7348,0.993213
1,Kyren Williams,0.7529,0.1507,0.0,0.9036,1.000000
2,Travis Kelce,0.0000,0.5385,0.0,0.5385,0.959276
3,Saquon Barkley,0.6461,0.1193,0.0,0.7654,0.995475
4,Amon-Ra St. Brown,0.0065,0.5287,0.0,0.5352,0.957014


In [86]:
# Fuzzy Merge odds and projections
odds_df['name'] = odds_df['name'].str.lower()
projections['playerName'] = projections['playerName'].str.lower()
merged = odds_df.merge(projections, how='outer', left_on='name', right_on='playerName')

# Difference between percentile_x and percentile_y
merged['percentile_diff'] = merged['odds_percentile'] - merged['proj_percentile']

merged = merged[['name', 'odds', 'probability', 'totalTd', 'percentile_diff']]
merged.sort_values(by='percentile_diff', ascending=True).reset_index(drop=True).head(30)

,name,odds,probability,totalTd,percentile_diff
0,julius chestnut,1000.0,0.090909,0.2303,-0.459432
1,ronnie rivers,1300.0,0.071429,0.1492,-0.419488
2,trey benson,1100.0,0.083333,0.1645,-0.404743
3,noah gray,950.0,0.095238,0.1879,-0.360275
4,luke farrell,2300.0,0.041667,0.0601,-0.356998
5,eric saubert,1600.0,0.058824,0.0837,-0.353955
6,josh whyle,1700.0,0.055556,0.0710,-0.332735
7,john metchie iii,950.0,0.095238,0.1560,-0.312763
8,braxton berrios,1700.0,0.055556,0.0652,-0.307848
9,craig reynolds,2600.0,0.037037,0.0408,-0.306366
